In [0]:
%pip install bokeh

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
file_clinic = '/FileStore/tables/clinicaltrial_2021.csv'
file_pharma = '/FileStore/tables/pharma.csv'
file_mesh = '/FileStore/tables/mesh.csv'
Year = '2021'

#####Task 1: Counting the Number of studies in the dataset

In [0]:
myRDD = sc.textFile(file_clinic)
myRDD.collect()[:10]

Out[2]: ['Id|Sponsor|Status|Start|Completion|Type|Submission|Conditions|Interventions',
 'NCT02758028|The University of Hong Kong|Recruiting|Aug 2005|Nov 2021|Interventional|Apr 2016||',
 'NCT02751957|Duke University|Completed|Jul 2016|Jul 2020|Interventional|Apr 2016|Autistic Disorder,Autism Spectrum Disorder|',
 'NCT02758483|Universidade Federal do Rio de Janeiro|Completed|Mar 2017|Jan 2018|Interventional|Apr 2016|Diabetes Mellitus|',
 'NCT02759848|Istanbul Medeniyet University|Completed|Jan 2012|Dec 2014|Observational|May 2016|Tuberculosis,Lung Diseases,Pulmonary Disease|',
 'NCT02758860|University of Roma La Sapienza|Active, not recruiting|Jun 2016|Sep 2020|Observational [Patient Registry]|Apr 2016|Diverticular Diseases,Diverticulum,Diverticulosis|',
 'NCT02757209|Consorzio Futuro in Ricerca|Completed|Apr 2016|Jan 2018|Interventional|Apr 2016|Asthma|Fluticasone,Xhance,Budesonide,Formoterol Fumarate,Salmeterol Xinafoate',
 'NCT02752438|Ankara University|Unknown status|May 2016|Jul 2

In [0]:
### Removing the header of the dataset
header = myRDD.first()
myRDD = myRDD.filter(lambda arg: arg!= header)
myRDD.take(10)

Out[3]: ['NCT02758028|The University of Hong Kong|Recruiting|Aug 2005|Nov 2021|Interventional|Apr 2016||',
 'NCT02751957|Duke University|Completed|Jul 2016|Jul 2020|Interventional|Apr 2016|Autistic Disorder,Autism Spectrum Disorder|',
 'NCT02758483|Universidade Federal do Rio de Janeiro|Completed|Mar 2017|Jan 2018|Interventional|Apr 2016|Diabetes Mellitus|',
 'NCT02759848|Istanbul Medeniyet University|Completed|Jan 2012|Dec 2014|Observational|May 2016|Tuberculosis,Lung Diseases,Pulmonary Disease|',
 'NCT02758860|University of Roma La Sapienza|Active, not recruiting|Jun 2016|Sep 2020|Observational [Patient Registry]|Apr 2016|Diverticular Diseases,Diverticulum,Diverticulosis|',
 'NCT02757209|Consorzio Futuro in Ricerca|Completed|Apr 2016|Jan 2018|Interventional|Apr 2016|Asthma|Fluticasone,Xhance,Budesonide,Formoterol Fumarate,Salmeterol Xinafoate',
 'NCT02752438|Ankara University|Unknown status|May 2016|Jul 2017|Observational [Patient Registry]|Apr 2016|Hypoventilation|',
 'NCT02753543|R

In [0]:
splitRDD = myRDD.map(lambda arg: arg.split("|"))
splitRDD.take(2)

Out[4]: [['NCT02758028',
  'The University of Hong Kong',
  'Recruiting',
  'Aug 2005',
  'Nov 2021',
  'Interventional',
  'Apr 2016',
  '',
  ''],
 ['NCT02751957',
  'Duke University',
  'Completed',
  'Jul 2016',
  'Jul 2020',
  'Interventional',
  'Apr 2016',
  'Autistic Disorder,Autism Spectrum Disorder',
  '']]

In [0]:
RDD_filter = splitRDD.map(lambda arg: arg[5])
RDD_filter.count()

Out[5]: 387261

#####Task 2: List all the types of studies with their frequencies ordered in Descending

In [0]:
RDD_filter = splitRDD.map(lambda arg: arg[5])\
            .map(lambda arg: (arg, 1))\
            .reduceByKey(lambda arg1, arg2: arg1+arg2)\
            .sortBy(lambda arg: arg[1], False)
RDD_filter.collect()

Out[6]: [('Interventional', 301472),
 ('Observational', 77540),
 ('Observational [Patient Registry]', 8180),
 ('Expanded Access', 69)]

#####Task 3: List the top 5 conditions with their frequencies

In [0]:
conditions = splitRDD.map(lambda arg: arg[7])\
            .filter(lambda arg: (arg != ""))\
            .flatMap(lambda arg: arg.split(","))\
            .map(lambda arg: (arg, 1))\
            .reduceByKey(lambda arg1, arg2: arg1+arg2)\
            .sortBy(lambda arg: arg[1], False)
conditions.take(5)
#conditions.collect()[3370:3389]

Out[7]: [('Carcinoma', 13389),
 ('Diabetes Mellitus', 11080),
 ('Neoplasms', 9371),
 ('Breast Neoplasms', 8640),
 ('Syndrome', 8032)]

#####Task 4: Finding the 5 most frequest roots

In [0]:
# This code imports the mesh file into a variable called mesh
mesh = sc.textFile(file_mesh)
mesh.take(20)

Out[8]: ['term,tree',
 'Calcimycin,D03.633.100.221.173',
 'A-23187,D03.633.100.221.173',
 'Temefos,D02.705.400.625.800',
 'Temefos,D02.705.539.345.800',
 'Temefos,D02.886.300.692.800',
 'Abate,D02.705.400.625.800',
 'Abate,D02.705.539.345.800',
 'Abate,D02.886.300.692.800',
 'Difos,D02.705.400.625.800',
 'Difos,D02.705.539.345.800',
 'Difos,D02.886.300.692.800',
 'Abattoirs,J01.576.423.200.700.100',
 'Abattoirs,J03.540.020',
 'Abbreviations as Topic,L01.559.598.400.556.131',
 'Acronyms as Topic,L01.559.598.400.556.131',
 'Abdomen,A01.923.047',
 '"Abdomen, Acute",C23.888.592.612.054.200',
 '"Abdomen, Acute",C23.888.821.030.249',
 'Abdominal Injuries,C26.017']

In [0]:
#Various data cleaning techniques were implemented in this block to isolate the term and the first node of the tree
mesh_split = mesh.map(lambda arg: arg.replace('"',''))\
            .map(lambda arg: arg.replace(', ',' '))\
            .map(lambda arg: arg.replace('.',','))\
            .map(lambda arg: arg.split(","))
mesh_split.take(10)

Out[9]: [['term', 'tree'],
 ['Calcimycin', 'D03', '633', '100', '221', '173'],
 ['A-23187', 'D03', '633', '100', '221', '173'],
 ['Temefos', 'D02', '705', '400', '625', '800'],
 ['Temefos', 'D02', '705', '539', '345', '800'],
 ['Temefos', 'D02', '886', '300', '692', '800'],
 ['Abate', 'D02', '705', '400', '625', '800'],
 ['Abate', 'D02', '705', '539', '345', '800'],
 ['Abate', 'D02', '886', '300', '692', '800'],
 ['Difos', 'D02', '705', '400', '625', '800']]

In [0]:
# This code presents the isolated diseases and the first node of the tree stored in a variable names mesh_roots
mesh_roots = mesh_split.map(lambda arg: (arg[0], arg[1]))
mesh_roots.collect()[:15]

Out[10]: [('term', 'tree'),
 ('Calcimycin', 'D03'),
 ('A-23187', 'D03'),
 ('Temefos', 'D02'),
 ('Temefos', 'D02'),
 ('Temefos', 'D02'),
 ('Abate', 'D02'),
 ('Abate', 'D02'),
 ('Abate', 'D02'),
 ('Difos', 'D02'),
 ('Difos', 'D02'),
 ('Difos', 'D02'),
 ('Abattoirs', 'J01'),
 ('Abattoirs', 'J03'),
 ('Abbreviations as Topic', 'L01')]

In [0]:
#mesh_roots data was joined with the diseases saved in variable named conditions and reduced to count the individual tree nodes and also sorted to identify the 5 most frequest roots.
freq_roots = mesh_roots.join(conditions)\
            .map(lambda arg: arg[1])\
            .reduceByKey(lambda arg1, arg2: arg1+arg2)\
            .sortBy(lambda arg: arg[1], False)
freq_roots.take(5)

Out[11]: [('C04', 143994),
 ('C23', 136079),
 ('C01', 106674),
 ('C14', 94523),
 ('C10', 92310)]

In [0]:
tree = freq_roots.map(lambda arg: arg[0]).collect()[:5]
count = freq_roots.map(lambda arg: arg[1]).collect()[:5]

In [0]:
from bokeh.plotting import figure, output_file, show
from bokeh.embed import file_html
from bokeh.resources import CDN
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.palettes import Spectral5

my_data = {
    "tree": tree,
    "count": count
}
rootViz = figure(x_range=tree, 
           height=250, 
           title="TreeRoot counts", 
           toolbar_location=None, 
           tools="hover")

rootViz.vbar(x=tree, top=count, width=0.8, color=Spectral5)

rootViz.xgrid.grid_line_color = None
#rootViz.legend.orientation = "horizontal"
#rootViz.legend.location = "top_center"

html = file_html(rootViz, CDN, "my_data")
displayHTML(html)

<!DOCTYPE html>
 
 
 
 
 
 my_data

#####Task 5: Finding the 10 most common sponsors that are NOT pharmaceutical companies

In [0]:
# This code imports the pharm file into a variable called pharm
pharma = sc.textFile(file_pharma)
pharma.take(2)

Out[14]: ['"Company","Parent_Company","Penalty_Amount","Subtraction_From_Penalty","Penalty_Amount_Adjusted_For_Eliminating_Multiple_Counting","Penalty_Year","Penalty_Date","Offense_Group","Primary_Offense","Secondary_Offense","Description","Level_of_Government","Action_Type","Agency","Civil/Criminal","Prosecution_Agreement","Court","Case_ID","Private_Litigation_Case_Title","Lawsuit_Resolution","Facility_State","City","Address","Zip","NAICS_Code","NAICS_Translation","HQ_Country_of_Parent","HQ_State_of_Parent","Ownership_Structure","Parent_Company_Stock_Ticker","Major_Industry_of_Parent","Specific_Industry_of_Parent","Info_Source","Notes"',
 '"Abbott Laboratories","Abbott Laboratories","$5,475,000","$0","$5,475,000","2013","20131227","government-contracting-related offenses","False Claims Act and related","kickbacks and bribery","Abbott Laboratories agreed to $5.475 million to resolve allegations that it violated the False Claims Act by paying kickbacks to induce doctors to implant the c

In [0]:
firstRow = pharma.first()
pharma = pharma.filter(lambda arg: arg!= firstRow)
pharma.take(20)

Out[15]: ['"Abbott Laboratories","Abbott Laboratories","$5,475,000","$0","$5,475,000","2013","20131227","government-contracting-related offenses","False Claims Act and related","kickbacks and bribery","Abbott Laboratories agreed to $5.475 million to resolve allegations that it violated the False Claims Act by paying kickbacks to induce doctors to implant the company\'s carotid, biliary and peripheral vascular products.","federal","agency action","Justice Department Civil Division","civil","","","","","","","","","","","","USA","Illinois","publicly traded","ABT","pharmaceuticals","pharmaceuticals","https://www.justice.gov/opa/pr/abbott-laboratories-pays-us-5475-million-settle-claims-company-paid-kickbacks-physicians",""',
 '"Abbott Laboratories Inc.","AbbVie","$1,500,000,000","$0","$1,500,000,000","2012","20120507","healthcare-related offenses","off-label or unapproved promotion of medical products","","Global Health Care Company Abbott Laboratories Inc. has pleaded guilty and agreed to

In [0]:
#Data Cleaning, split and grouping
pharma_clean = pharma.map(lambda arg: arg.replace(', ', ''))\
               .map(lambda arg: arg.replace('"', ''))\
               .map(lambda arg: arg.split(","))\
               .map(lambda arg: (arg[1], 1))
pharma_clean.take(10)

Out[16]: [('Abbott Laboratories', 1),
 ('AbbVie', 1),
 ('AbbVie', 1),
 ('Abbott Laboratories', 1),
 ('Johnson & Johnson', 1),
 ('Abbott Laboratories', 1),
 ('Abbott Laboratories', 1),
 ('Johnson & Johnson', 1),
 ('Johnson & Johnson', 1),
 ('Abbott Laboratories', 1)]

In [0]:
sponsors = splitRDD.map(lambda arg: (arg[1], 1))\
           .reduceByKey(lambda arg1, arg2: arg1 + arg2)\
           .sortBy(lambda arg: arg[1], False)
sponsors.take(10)

Out[17]: [('GlaxoSmithKline', 3378),
 ('National Cancer Institute (NCI)', 3218),
 ('AstraZeneca', 2691),
 ('Pfizer', 2645),
 ('M.D. Anderson Cancer Center', 2414),
 ('Assistance Publique - Hôpitaux de Paris', 2369),
 ('Mayo Clinic', 2300),
 ('Merck Sharp & Dohme Corp.', 2243),
 ('Assiut University', 2154),
 ('Novartis Pharmaceuticals', 2088)]

In [0]:
common_sponsors = sponsors.subtractByKey(pharma_clean)\
                  .sortBy(lambda arg: arg[1], False)
common_sponsors.take(10)

Out[18]: [('National Cancer Institute (NCI)', 3218),
 ('M.D. Anderson Cancer Center', 2414),
 ('Assistance Publique - Hôpitaux de Paris', 2369),
 ('Mayo Clinic', 2300),
 ('Merck Sharp & Dohme Corp.', 2243),
 ('Assiut University', 2154),
 ('Novartis Pharmaceuticals', 2088),
 ('Massachusetts General Hospital', 1971),
 ('Cairo University', 1928),
 ('Hoffmann-La Roche', 1828)]

#####Task 6: Visualizing completed studies per month

In [0]:
studyPerMonth = splitRDD.map(lambda arg: (arg[2], arg[4]))\
            .filter(lambda arg: arg[0] == 'Completed')\
            .map(lambda arg: (arg[1]))\
            .map(lambda arg: (arg.split(' ')))\
            .map(lambda arg: arg)\
            .filter(lambda arg: arg[-1] == Year)\
            .map(lambda arg: (arg[0], 1))\
            .reduceByKey(lambda arg1, arg2: arg1 + arg2)
studyPerMonth.collect()

Out[19]: [('May', 984),
 ('Jan', 1131),
 ('Jun', 1094),
 ('Mar', 1227),
 ('Feb', 934),
 ('Aug', 700),
 ('Apr', 967),
 ('Jul', 819),
 ('Oct', 187),
 ('Sep', 528)]

In [0]:
#This function sorts the study result in ascending order of Months
def sortFunction(arg):
    if arg[0]=='Jan':
        result=(1, arg)
    elif arg[0]=='Feb':
        result=(2, arg)
    elif arg[0]=='Mar':
        result=(3, arg)
    elif arg[0]=='Apr':
        result=(4, arg)
    elif arg[0]=='May':
        result=(5, arg)
    elif arg[0]=='Jun':
        result=(6, arg)
    elif arg[0]=='Jul':
        result=(7, arg)
    elif arg[0]=='Aug':
        result=(8, arg)
    elif arg[0]=='Sep':
        result=(9, arg)
    elif arg[0]=='Oct':
        result=(10, arg)
    elif arg[0]=='Nov':
        result=(11, arg)
    else:
        result=(12, arg)
    return result
studyPlot = studyPerMonth.map(lambda arg: sortFunction(arg)).sortBy(lambda arg: arg[0], True).map(lambda arg: arg[1])
studyPlot.collect()

Out[20]: [('Jan', 1131),
 ('Feb', 934),
 ('Mar', 1227),
 ('Apr', 967),
 ('May', 984),
 ('Jun', 1094),
 ('Jul', 819),
 ('Aug', 700),
 ('Sep', 528),
 ('Oct', 187)]

In [0]:
months = studyPlot.map(lambda arg: arg[0]).collect()
studies = studyPlot.map(lambda arg: arg[1]).collect()
studies


Out[21]: [1131, 934, 1227, 967, 984, 1094, 819, 700, 528, 187]

In [0]:
from bokeh.plotting import figure, output_file, show
from bokeh.embed import file_html
from bokeh.resources import CDN
from bokeh.models import ColumnDataSource, HoverTool
 
my_data = {
    "months": months,
    "studies": studies
}
cs = figure(x_range = months, 
           height = 300, 
           title = "Completed Studies per Month",
           toolbar_location = None,
           tools = "hover"
          )
cs.vbar(x = months,
       top = studies, 
       width = 0.9
      )

cs.xgrid.grid_line_color = None
cs.y_range.start = 0
html = file_html(cs, CDN, "my_data")
displayHTML(html)

<!DOCTYPE html>
 
 
 
 
 
 my_data

####Further Analysis 1: Popular Interventions given to each Condition

In [0]:
#Top 5 conditions in the clinical trial
splitRDD.map(lambda arg: arg[7])\
            .filter(lambda arg: (arg != ""))\
            .flatMap(lambda arg: arg.split(","))\
            .map(lambda arg: (arg, 1))\
            .reduceByKey(lambda arg1, arg2: arg1+arg2)\
            .sortBy(lambda arg: arg[1], False).take(5)
#conditions.collect()[3370:3389]

Out[23]: [('Carcinoma', 13389),
 ('Diabetes Mellitus', 11080),
 ('Neoplasms', 9371),
 ('Breast Neoplasms', 8640),
 ('Syndrome', 8032)]

In [0]:
#Knowing the top 5 conditions in the clinical trial, we can find the most common Intervention given to patients with one of the top 5 conditions('Diabetes Mellitus'). It was observed that Insulin is the most common interventions given to patients with Diabetes Mellitus.
splitRDD.map(lambda arg: (arg[7], arg[8]))\
.filter(lambda arg: ('Diabetes Mellitus' in arg[0]))\
.flatMap(lambda arg: arg[1].split(","))\
.filter(lambda arg: (arg != ""))\
.map(lambda arg: (arg, 1))\
.reduceByKey(lambda arg1, arg2: arg1+arg2)\
.sortBy(lambda arg: arg[1], False)\
.take(5)

Out[24]: [('Insulin', 1104),
 ('Metformin', 747),
 ('Insulin Glargine', 426),
 ('Insulin Aspart', 372),
 ('Sitagliptin Phosphate', 330)]